<h1><center>Obtaining Stocks and Market Data + Fama-MacBeth Regression</center></h1>

## Introduction to Programming 2477-2223 S1
## Nova School of Business and Economics


### Team: Daniel Brosko (55036), ... (TODO)

TODO: theoretical description

## 1 Introduction

TODO

## 2 General packages loading

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time
import yfinance as yf
from sklearn.linear_model import LinearRegression

#importing own functions
import scraping_functions as sf
import computing_functions as cf
import stock_prices as sp


In [2]:
# pip list # used for requirements.txt to get the version numbers

In [3]:
# loading tickers of selected stocks
tickers = sf.load_tickers("tickers_set.csv")

## 3 Obtaining the Data
### 3.1 Market Capitalization (Size) Data Scraping

Here we scrape the market capitalizations of the firms using own scraping function from scraping_function.py.
The data are in billions of USD.

In [4]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(sf.mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Elapsed time:', round(elapsed_time,3) , 'seconds.')

# mkt_cap_df_list

Elapsed time: 5.185 seconds.


In [5]:
# merging mkt. capital for all stocks, and renaming the columns according to particular stocks
df_mkt_cap = mkt_cap_df_list[0]

for dataframe in mkt_cap_df_list[1:] :
    df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')

mc_columns_names = ['Date']
mc_columns_names.extend(tickers)
df_mkt_cap.columns = mc_columns_names
df_mkt_cap.set_index('Date', inplace=True)

# using size instead of mkt_cap to rule out skewness
df_size = np.log(df_mkt_cap + 0.000001)
df_size

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_80621/1964478080.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_80621/1964478080.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_80621/1964478080.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_80621/1964478080.py:5: 

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2009-04-01,-0.248460,-13.815511,1.830980,0.712950,0.609766,0.067660,3.455686,1.605430,1.570697,2.517697,3.307985,2.600465,2.210470,0.157005,2.706048,1.834180,4.544571,1.902108,0.657521,2.030777
2009-04-02,-0.174352,-13.815511,1.887070,0.727549,0.524729,0.104361,3.493473,1.682689,1.587193,2.550226,3.352007,2.614472,2.157559,0.207015,2.712706,1.932970,4.528721,1.955861,0.693148,2.089392
2009-04-03,-0.116533,-13.815511,1.864080,0.756122,0.582216,0.095311,3.517201,1.708378,1.601406,2.536866,3.376563,2.607862,2.170196,0.239018,2.675527,1.947338,4.512616,1.981002,0.727549,2.094330
2009-04-06,-0.127832,-13.815511,1.843719,0.770109,0.506818,0.086179,3.514824,1.699279,1.609438,2.545531,3.329701,2.593761,2.173615,0.190621,2.661657,1.934416,4.524394,1.953028,0.657521,2.074429
2009-04-07,-0.186328,-13.815511,1.783391,0.760806,0.582216,0.058270,3.482470,1.688249,1.581039,2.532108,3.274500,2.588516,2.140066,0.157005,2.710048,1.888584,4.509760,1.905088,0.587787,2.025513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,2.681706,4.001864,5.246550,3.568123,1.888584,2.859913,6.891463,3.007167,4.002229,4.171460,4.211387,3.163363,3.948548,1.497389,3.785779,4.176232,5.638639,3.744787,3.221273,2.528126
2022-12-01,2.669309,3.936325,5.248864,3.598955,1.882514,2.890927,6.880631,3.011113,3.993603,4.190109,4.210497,3.163786,3.938275,1.589235,3.774828,4.175617,5.657424,3.751150,3.205183,2.512846
2022-12-02,2.673459,3.940610,5.252797,3.590164,1.885554,2.900322,6.866183,3.017983,3.995077,4.200355,4.216857,3.160399,3.945651,1.581039,3.764914,4.175310,5.654067,3.752089,3.190476,2.492379


### 3.2 Obtaining Stock Prices

In [6]:
# downloading stocks prices - designed to save as csv, so we have "back-up" if something goes wrong later
success_tickers, failed_tickers = sp.get_stocks(tickers)

TER - SUCCESS
ADM - SUCCESS
TMUS - SUCCESS
IDXX - SUCCESS
LNC - SUCCESS
WST - SUCCESS
AMZN - SUCCESS
DOV - SUCCESS
ORLY - SUCCESS
BSX - SUCCESS
USB - SUCCESS
ETR - SUCCESS
AZO - SUCCESS
PVH - SUCCESS
BIIB - SUCCESS
ETN - SUCCESS
PFE - SUCCESS
WMB - SUCCESS
FITB - SUCCESS
CHRW - SUCCESS
Elapsed time: 17.14 seconds.


In [7]:
# loading stock prices into list of dataframes using own function
stocks_df_list = sp.load_stocks(success_tickers)
# stocks_df_list

In [8]:
df_close_prices = stocks_df_list[0][['Date','Adj Close']]
 
for index in range(1, len(stocks_df_list)):
    dataframe = stocks_df_list[index][['Date','Adj Close']]
    df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')

cp_columns_names = ['Date']
cp_columns_names.extend(success_tickers)
df_close_prices.columns = cp_columns_names
df_close_prices['Date'] = pd.to_datetime(df_close_prices['Date']).dt.date
df_close_prices.set_index('Date', inplace=True)

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_80621/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_80621/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_80621/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn

In [9]:
# Making returns from daily prices
# daily_returns = df_close_prices.pct_change(1) # 1 for ONE DAY lookback
daily_returns = np.log(df_close_prices/df_close_prices.shift())
daily_returns.dropna(inplace=True)
daily_returns.head()

# monthly_return = prices.pct_change(21) # 21 for ONE MONTH lookback
# annual_return = prices.pct_change(252)

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2009-12-09,0.011288,-0.010769,-0.026550,-0.005999,0.007538,-0.002321,-0.021099,-0.005363,-0.008706,0.000000,-0.003487,-0.000494,-0.003767,-0.018150,0.002286,0.000000,0.026668,0.000517,0.009765,0.000870
2009-12-10,-0.015424,0.000983,0.030908,0.009658,0.005286,0.006947,0.030525,0.001954,-0.003451,0.027909,0.005660,0.010699,-0.005839,0.007097,0.001867,0.005138,0.004377,0.013864,-0.020619,0.003995
2009-12-11,-0.003114,-0.000656,0.007220,0.002688,-0.014159,0.013245,-0.009127,0.001949,-0.001064,-0.005750,0.000868,0.015898,0.002442,0.022231,0.008049,0.001242,-0.001092,-0.001021,0.009872,0.007081
2009-12-14,0.069270,-0.008564,-0.011578,0.006878,0.012841,0.009820,-0.020865,0.012823,0.009273,0.029548,-0.000434,0.008392,0.017688,0.009099,0.004512,0.019355,0.005450,0.036587,0.014627,0.016050
2009-12-15,0.002906,0.013471,0.067536,0.008343,-0.027203,0.001502,-0.008792,-0.005786,-0.010604,-0.007870,-0.028617,-0.009115,-0.010461,-0.012068,0.011394,-0.001523,-0.005996,0.013199,-0.052669,0.011285


### 3.3 Market Return from prof. Kenneth French's website 

We dowloaded data about the market from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html , however, we need to change the date into suitable format.

In [10]:
data_market = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=3)
data_market = data_market[:-1]

data_market.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
data_market['Date'] = data_market['Date'].map(date_format_lambda)
data_market['Date'] = pd.to_datetime(data_market['Date']).dt.date
data_market.set_index('Date', inplace=True)
data_market.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009


### 3.4 Short-Time-Reversal from prof. Kenneth French's website 
We dowloaded STR data from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
and proceed in the similar fashion to subsection 3.3.

In [11]:
st_reversal = pd.read_csv("F-F_ST_Reversal_Factor_daily.CSV", skiprows=12)
st_reversal = st_reversal[:-1]
st_reversal.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
st_reversal['Date'] = st_reversal['Date'].map(date_format_lambda)
st_reversal['Date'] = pd.to_datetime(st_reversal['Date']).dt.date
st_reversal.set_index('Date', inplace=True)
st_reversal.head()

,ST_Rev
Date,
1926-01-26,0.14
1926-01-27,0.62
1926-01-28,0.22
1926-01-29,0.58
1926-01-30,0.46


### 3.4 Filtering data for reasonable time interval

In [12]:
size = df_size.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
returns = daily_returns.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
market = data_market.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
reversal = st_reversal.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")

In [13]:
# adjusting stock returns to risk-free rate
risk_free_rate = market.loc[:, 'RF']
risk_free_rate = pd.DataFrame(risk_free_rate)
risk_free_rate_column = risk_free_rate.copy()

for column in range(1, len(success_tickers)):
    risk_free_rate = pd.concat([risk_free_rate, risk_free_rate_column], axis=1)

excess_returns = returns.subtract(risk_free_rate.values)

excess_returns

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.021209,0.005097,0.043597,0.011718,0.039406,0.006864,-0.004620,0.017864,0.009919,0.001111,0.016304,0.003781,-0.000253,0.010272,0.002613,0.010943,0.039876,0.024831,0.037246,0.010333
2010-01-05,0.003643,0.005388,0.002506,0.001663,0.020651,-0.012749,0.005883,-0.004733,0.001557,0.003324,0.025033,-0.016075,-0.008388,0.036784,-0.004859,-0.005926,-0.014366,0.012417,0.026330,-0.020946
2010-01-06,-0.009133,-0.002532,-0.069977,0.002397,0.015026,-0.009799,-0.018282,0.011790,-0.004419,0.013187,0.010600,0.005059,-0.009425,0.023640,0.000936,-0.004389,-0.003220,0.031934,0.005758,-0.013339
2010-01-07,0.005490,-0.010511,0.009352,0.037235,0.023592,0.003104,-0.017160,0.021795,-0.003131,-0.007671,0.019216,-0.012011,0.013182,0.000000,-0.008269,0.014658,-0.003770,-0.007108,0.058540,-0.009638
2010-01-08,0.010889,-0.012566,-0.057471,-0.008016,-0.015419,-0.000258,0.026717,0.028267,-0.008924,-0.009950,0.001653,-0.009638,-0.012925,-0.001605,0.020361,0.022198,0.008062,0.007993,0.000902,0.013294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.016880,0.009619,-0.001641,0.010255,0.013569,0.040840,-0.004511,-0.002004,-0.001584,-0.003222,0.010170,0.012276,0.002356,0.047404,-0.014867,0.020155,-0.009903,0.012830,0.017114,-0.007195
2022-10-26,0.021121,0.015527,-0.009508,0.005746,-0.027826,0.009612,-0.052824,-0.008881,-0.008772,0.000795,-0.001995,-0.011286,-0.017666,-0.004292,0.013803,-0.014635,-0.000743,-0.001625,-0.006484,0.006774
2022-10-27,-0.040488,-0.005247,-0.012492,-0.017047,-0.000641,-0.150705,-0.052485,-0.007322,0.033969,-0.016038,-0.018340,-0.008044,0.028674,-0.027370,-0.020171,-0.002992,-0.017972,-0.000172,-0.015516,-0.007896


Just to remind what objects (dataframes) are important fot us now, we obtained the following dataframes that we are going to use:

* size - logarithm of market capitalization    
* excess_returns - daily stock returns adjusted to risk-free rate
* market - FF market factors

## 4 Computations
### 4.1 Estimating Market Betas

We estimate stocks's sensitivity to the market (beta factors) on daily data using linear regression (implemented in the market_betas function from computation_functions.py script).

In [14]:
betas = cf.market_betas(excess_returns, market)
betas.head()

Elapsed time: 15.005 seconds.


,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2011-01-04,0.017935,0.007380,0.011974,0.009055,0.018687,0.009052,0.010824,0.013625,0.007077,0.009581,0.011858,0.006959,0.004700,0.015762,0.007382,0.012936,0.007492,0.014149,0.017356,0.008058
2011-01-05,0.017977,0.007416,0.011807,0.009080,0.018637,0.009105,0.010932,0.013653,0.007119,0.009666,0.011879,0.006986,0.004768,0.015861,0.007439,0.012991,0.007343,0.014147,0.017316,0.008082
2011-01-06,0.017931,0.007414,0.011838,0.009078,0.018613,0.009118,0.010941,0.013630,0.007117,0.009667,0.011852,0.006981,0.004751,0.015846,0.007445,0.012999,0.007362,0.014128,0.017306,0.008083
2011-01-07,0.017933,0.007398,0.011895,0.009077,0.018613,0.009125,0.010952,0.013630,0.007133,0.009669,0.011856,0.006982,0.004765,0.015847,0.007441,0.013006,0.007361,0.014116,0.017313,0.008089
2011-01-10,0.017924,0.007408,0.011905,0.009041,0.018591,0.009127,0.010977,0.013611,0.007139,0.009693,0.011842,0.006996,0.004748,0.015876,0.007452,0.012994,0.007363,0.014120,0.017248,0.008104


### 4.2 Transforming Data to Monthly Values

In [15]:
# firstly restricting to same time range, since to compute betas, we needed to give up of some data
size = size.iloc[253:, :]
excess_returns = excess_returns.iloc[253:, :]
market = market.iloc[253:, :]
reversal = reversal.iloc[253:, :]

In [36]:
# to monthly
monthly_size = sp.daily_to_monthly(size)
monthly_returns = sp.daily_to_monthly(excess_returns)
monthly_market = sp.daily_to_monthly(market)
monthly_reversal = sp.daily_to_monthly(reversal)
monthly_betas = sp.daily_to_monthly(betas)

### 4.3 TODO Fama-MacBeth

In [37]:
monthly_returns_index = monthly_returns.index.copy() # saving the index

monthly_returns.reset_index(inplace=True)
monthly_betas.reset_index(inplace=True)
monthly_returns.drop(['Date'], axis=1, inplace=True)
monthly_betas.drop(['Date'], axis=1, inplace=True)

gammas = monthly_returns.copy()
gammas.loc[:] = np.nan

for row in range(len(monthly_returns.index)):
    y = monthly_returns.iloc[row, :].values.reshape(-1, 1)
    x = monthly_betas.iloc[row, :].values.reshape(-1, 1)
    model = LinearRegression(fit_intercept=False).fit(x, y)
    gammas.iloc[row, 0] = model.coef_
    
gammas.set_index(monthly_returns_index, inplace=True)
gammas = gammas.iloc[:, 0]
gammas.columns = ['Gamma']

monthly_returns.set_index(monthly_returns_index, inplace=True)
monthly_betas.set_index(monthly_returns_index, inplace=True)

gammas

Date
2011-01-31    0.045089
2011-02-28    0.110009
2011-03-31    0.039667
2011-04-30    0.117344
2011-05-31   -0.083393
                ...   
2022-06-30   -0.808659
2022-07-31    0.089913
2022-08-31   -1.064309
2022-09-30   -1.206335
2022-10-31   -0.651564
Freq: M, Name: TER, Length: 142, dtype: float64

In [23]:
STOP HERE
monthly_returns_index = monthly_returns.index.copy() # saving the index

monthly_returns.reset_index(inplace=True)
monthly_betas.reset_index(inplace=True)
excess_returns.drop(['Date'], axis=1, inplace=True)
excess_betas.drop(['Date'], axis=1, inplace=True)

betas = betas_returns.copy()
betas.loc[:] = np.nan

column = 0
for stock in excess_returns.columns:
    for row in range(len(excess_returns[stock])-253):
        y = excess_returns.iloc[row:(row + 253), :]
        y = y.loc[:, stock].values.reshape(-1, 1)
        x = market.iloc[row:(row + 253), :]
        x = x.loc[:,'Mkt-RF'].values.reshape(-1, 1)

        model = LinearRegression(fit_intercept=False).fit(x, y)
        betas.iloc[row + 253,column] = model.coef_
    column += 1

    #setting index back to date
excess_returns.set_index(excess_returns_index, inplace=True)
betas.set_index(excess_returns_index, inplace=True)

betas = betas.iloc[253:, :] # cutting off NAs

SyntaxError: invalid syntax (955619637.py, line 1)

In [ ]:
# TODO - regression - second step

# TODO - comment on intro/theory/findings + conclusion

# TODO - short readme update at the end

### 4.2 Estimating... TODO

In [ ]:
# TODO - download book to market ratio

In [ ]:
# TODO - perform regression

In [ ]:
# TODO - put results into table

In [ ]:
# TODO - comment on findings

In [ ]:
# todo - solve issues

tickers = pd.read_csv("tickers_set.csv", sep = ",")
tickers = pd.DataFrame(tickers)
tickers.columns = ["Index", "Ticker"]
# tickers.head()

tickers = tickers.Ticker
# tickers.head()

long_names = pd.DataFrame(index = range(len(tickers)), columns = 2)

for i in range(len(tickers)):
    long_names[i,0] = tickers[i]
    header = str(curlGetHeaders("https://www.macrotrends.net/stocks/charts/" + tickers[i])[5])
    long_names[i,1] = "Location: https://www.macrotrends.net/stocks/charts/" + tickers[i] + "/" + header
    long_names[i,1] = gsub("/\r\n","",long_names[i,2])
    
    
long_names

#stocks <- pd.Data.Frame()

In [ ]:
# url = https://www.macrotrends.net/stocks/charts/AAPL/apple/price-book
#page = requests.get('https://www.macrotrends.net/stocks/charts/AAPL')#/apple/price-book')


In [ ]:
my_date = datetime.strptime('2018-01-05', '%Y-%m-%d')
my_date.strftime('%A')